# VLSI - MIP Part
Di Tuccio Gianluca, Orsini Lorenzo

In [1]:
from tqdm import tqdm
from z3 import *
from numpy import random
import os
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os

random_seed = 42

'''
SET HERE THE PATH OF THE MAIN FOLDER (i.e VLSI folder)
            |
            |
            V
'''
os.chdir('/Users/gianlucadituccio/Documents/VLSI')
from utils.utils import *

## Results

In [2]:
results = pd.read_csv('utils/datasets/MIP.csv')
Statistics({'MIP w/out SB w/out R':results['MIP w/out SB w/out R'],
            'MIP + SB w/out R':results['MIP + SB w/out R'],
            'MIP w/out SB + R':results['MIP w/out SB + R'],
            'MIP + SB + R':results['MIP + SB + R']})
results = results.replace('TIMEOUT', 0)
results = results.astype(float)
results = results.reset_index(drop=True)
results = results.set_index("Instance")
print()
print('GREEN --> BEST VALUE,\nYELLOW --> WORST VALUE,\nRED --> TIMEOUT')
results.style.apply(highlight_max, axis=1).apply(highlight_min, axis=1).apply(highlight_timeout, axis=1).set_properties(**{'border-top': '2px solid', 'border-bottom': '2px solid'}, axis=1)

TIMEOUT is seen as 300s.

+------------------+----------------------+------------------+------------------+--------------+
|                  | MIP w/out SB w/out R | MIP + SB w/out R | MIP w/out SB + R | MIP + SB + R |
+------------------+----------------------+------------------+------------------+--------------+
|  Total Time [s]  |             2483.743 |         2504.815 |         4870.363 |     4838.492 |
|     Max [s]      |                300.0 |            300.0 |            300.0 |        300.0 |
|     Min [s]      |                0.002 |            0.002 |            0.002 |        0.002 |
|     Mean [s]     |               62.094 |            62.62 |          121.759 |      120.962 |
|     Std [s]      |              111.832 |          113.628 |          141.875 |      136.516 |
| Instances Solved |                34/40 |            33/40 |            26/40 |        27/40 |
+------------------+----------------------+------------------+------------------+--------------+

GRE

,MIP w/out SB w/out R,MIP + SB w/out R,MIP w/out SB + R,MIP + SB + R
Instance,,,,
1.0,0.004271,0.008457,0.002396,0.001950
2.0,0.002281,0.002499,0.002541,0.004301
3.0,0.006326,0.006792,0.001776,0.006159
4.0,0.006718,0.005207,0.009563,0.008328
5.0,0.010257,0.005924,0.051874,0.011513
6.0,0.003305,0.003089,0.088235,0.003880
7.0,0.072289,0.060265,0.075392,0.071345
8.0,0.014924,0.016494,0.016068,0.124616
9.0,0.022471,0.030247,0.043468,0.123509


## MIP without Rotation

In [ ]:
def MIP(parameters, timeout=300, simmetry=False, opt_focus=False):
    try:
        # declaration the variables for the current instance
        w = parameters['w']
        n = parameters['n_rectangles']
        
        # start the timer of the execution
        # the time for constraints is inglobed inside the timeout
        t_constraint = time.process_time()
        
        # sort rectangles by y component
        rectangles = [(x, y, i) for i, (x, y) in enumerate(zip(parameters['x_components'], parameters['y_components']))]
        sorted_rectangles = sorted(rectangles, key=lambda x: (x[1],x[0]), reverse=True)
        x_components = [k[0] for k in sorted_rectangles]
        y_components = [k[1] for k in sorted_rectangles]
        original_indices = [k[2] for k in sorted_rectangles]
        area_min = sum(x_components[i] * y_components[i] for i in range(n))
        h_min = area_min / w
        h_max = sum(y_components)
        M = h_max
        
        # declaration of the model
        model = gp.Model('MIP')
        #model.setParam("UserName", "gianluca.dituccio@studio.unibo.it")
        model.params.OutputFlag = 0 # hide inter-outputs
        model.setParam('TimeLimit', timeout)
        model.params.Seed = random_seed
        
        if simmetry:
            model.params.Symmetry = 2
        
        # 0: balanced search (default value), 1: feasibility focus, 2: optimality focus, 3: bound focus.
        if opt_focus:
            model.params.MIPFocus = 3

 
        # declaration of the variables for GUROBY
        x_positions = model.addVars(n, ub = w, vtype = GRB.INTEGER, name = 'x_positions')
        y_positions = model.addVars(n, ub = h_max, vtype = GRB.INTEGER, name = 'y_positions')
        OR_overlap = model.addVars(n, n, 4, vtype = GRB.BINARY, name = 'OR_overlap') # used for the OR of the overlap constraints
        height = model.addVar(vtype = GRB.INTEGER, name = 'height')
        
        # higher block in (0,0)
        model.addConstr(x_positions[0] == 0, 'x_0_0')
        model.addConstr(y_positions[0] == 0, 'y_0_0')
        
        # positions >= 0
        model.addConstrs((x_positions[i] >= 0 for i in range(n)), 'x_positive_coordinates_constraint')
        model.addConstrs((y_positions[i] >= 0 for i in range(n)), 'y_positive_coordinates_constraint')
        
        # height constraints
        model.addConstr(height >= h_min, 'height_min_constraint')
        model.addConstr(height <= h_max, 'height_max_constraint')
        
        # position + component <= plate width
        model.addConstrs(((x_positions[i] + x_components[i] <= w) for i in range(n)), 'width_constraint')
        model.addConstrs(((y_positions[i] + y_components[i] <= height) for i in range(n)), 'height_constraint')

        # non overlap constraints
        model.addConstrs(((x_positions[i] + x_components[i] <= x_positions[j] + M*OR_overlap[i,j,0]) for i in range(n) for j in range(i+1,n)), "or1")
        model.addConstrs(((y_positions[i] + y_components[i] <= y_positions[j] + M*OR_overlap[i,j,1]) for i in range(n) for j in range(i+1,n)), "or2")
        model.addConstrs(((x_positions[j] + x_components[j] <= x_positions[i] + M*OR_overlap[i,j,2]) for i in range(n) for j in range(i+1,n)), "or3")
        model.addConstrs(((y_positions[j] + y_components[j] <= y_positions[i] + M*OR_overlap[i,j,3]) for i in range(n) for j in range(i+1,n)), "or4")
        model.addConstrs((gp.quicksum(OR_overlap[i,j,k] for k in range(4)) <= 3 for i in range(n) for j in range(n)), 'no_overlap_constraint') 
        
        # objective function
        model.setObjective(height, GRB.MINIMIZE)
        
        t1 = time.time()
        model.optimize()
        t2 = time.time() - t1
        if t2>timeout:
            return {}
        
        x_positions = []
        y_positions = []

        for i in range(n):
            x_positions.append(int(model.getVarByName(f'x_positions[{i}]').X))
            y_positions.append(int(model.getVarByName(f'y_positions[{i}]').X))
        
        # re-take the original order of the rectangles
        positions = [(x, y, i, z, k) for x, y, i, z, k in zip(x_positions, y_positions, x_components, y_components, original_indices)]
        x_positions, y_positions, x_components, y_components, _ = zip(*sorted(positions, key=lambda x: x[4]))
            
        height_found = int(model.ObjVal)
        return {'x_components':x_components,
                'y_components':y_components,
                'x_positions':x_positions,
                'y_positions':y_positions,
                'h':height_found,
                'time':t2,
                'W':w}
        
    except Exception as ex: print(ex)

In [ ]:
for i in tqdm(range(1,41)):
    file = LoadInstance('utils/instances/',f'ins-{i}.txt')
    result = MIP(parameters=file, timeout=300, simmetry=True, opt_focus=False)
    if result:
        # if you want plot the results without saving them, use: PlotInstance(result)
        PlotInstance(result, 
                     path='MIP/images', 
                     name='fig-'+str(i))
        WriteInstance(result,
                     path='MIP/out',
                     name='out-'+str(i))
    else:
        print('Instance', i, 'TIMEOUT')

## MIP with Rotation
Here, we simply add two new array for storing the components when the rotation is enabled.

In [ ]:
def MIP(parameters, timeout=300, simmetry=False, opt_focus=False):
    try:
        # declaration the variables for the current instance
        w = parameters['w']
        n = parameters['n_rectangles']
        
        # start the timer of the execution
        # the time for constraints is inglobed inside the timeout
        t_constraint = time.process_time()

        # sort rectangles by y component and find the min/max height
        rectangles = [(x, y, i) for i, (x, y) in enumerate(zip(parameters['x_components'], parameters['y_components']))]
        sorted_rectangles = sorted(rectangles, key=lambda x: (x[1],x[0]), reverse=True)
        x_components = [k[0] for k in sorted_rectangles]
        y_components = [k[1] for k in sorted_rectangles]
        original_indices = [k[2] for k in sorted_rectangles] 
        area_min = sum(x_components[i]*y_components[i] for i in range(n))
        h_min = area_min / w
        h_max = sum(y_components)
        M = h_max
        
        # declaration of the model
        model = gp.Model('MIP')
        model.params.OutputFlag = 0 # hide inter-outputs
        model.setParam('TimeLimit', timeout)
        model.params.Seed = random_seed
        
        # declaration of the variables for GUROBY
        x_positions = model.addVars(n, ub=w, vtype=GRB.INTEGER, name='x_positions')
        y_positions = model.addVars(n, ub=h_max, vtype=GRB.INTEGER, name='y_positions')
        OR_overlap = model.addVars(n, n, 4, vtype=GRB.BINARY, name='OR_overlap') # used for the OR of the overlap constraints
        x_components_rotated = model.addVars(n, vtype=GRB.INTEGER, name='x_components_rotated')
        y_components_rotated = model.addVars(n, vtype=GRB.INTEGER, name='y_components_rotated')
        rot = model.addVars(n, vtype=GRB.BINARY, name='rotation') # 0 --> no rotation, 1 --> rotation
        height = model.addVar(vtype=GRB.INTEGER, name='height')
        
        
        if simmetry:
            # higher block in (0,0)
            model.addConstr(x_positions[0] == 0, 'x_0_0')
            model.addConstr(y_positions[0] == 0, 'y_0_0')
            model.addConstr(rot[0] == 0)
            
            # same components --> rotation set to False
            # rotation constraints (0 --> no rotation, 1 --> rotation)
            model.addConstrs(((rot[i] == 0) for i in range(n) if x_components[i] == y_components[i]))
            
            model.params.Symmetry = 2
            
        # if we want to rotate the rectangle, the x_component must be lower than the height of the plate
        model.addConstrs(((rot[i] == 0) for i in range(n) if x_components[i] > h_min))
        model.addConstrs(((rot[i] == 0) for i in range(n) if y_components[i] > w))
           
        
        # 0: balanced search (default value), 1: feasibility focus, 2: optimality focus, 3: bound focus.
        if opt_focus:
            model.params.MIPFocus = 2

        # positions >= 0
        model.addConstrs((x_positions[i] >= 0 for i in range(n)), 'x_positive_coordinates_constraint')
        model.addConstrs((y_positions[i] >= 0 for i in range(n)), 'y_positive_coordinates_constraint')
        
        # height constraints
        model.addConstr(height >= h_min, 'height_min_constraint')
        model.addConstr(height <= h_max, 'height_max_constraint')
        
        # rotation constraints (0 --> no rotation, 1 --> rotation)
        # if the rotation is set to 0, then assign y_components to y_components_rotated, otherwise assign the opposite components
        model.addConstrs(((y_components_rotated[i] == rot[i]*x_components[i] + (1-rot[i])*y_components[i]) for i in range(n)), 'x_rotation')
        model.addConstrs(((x_components_rotated[i] == rot[i]*y_components[i] + (1-rot[i])*x_components[i]) for i in range(n)), 'y_rotation')
        
        # position + component <= plate width
        model.addConstrs(((x_positions[i]+x_components_rotated[i] <= w) for i in range(n)), 'width_constraint')
        model.addConstrs(((y_positions[i]+y_components_rotated[i] <= height) for i in range(n)), 'height_constraint')

        # non overlap constraints
        model.addConstrs(((x_positions[i]+x_components_rotated[i] <= x_positions[j] + M*OR_overlap[i,j,0]) for i in range(n) for j in range(i+1,n)), "or1")
        model.addConstrs(((y_positions[i]+y_components_rotated[i] <= y_positions[j] + M*OR_overlap[i,j,1]) for i in range(n) for j in range(i+1,n)), "or2")
        model.addConstrs(((x_positions[j]+x_components_rotated[j] <= x_positions[i] + M*OR_overlap[i,j,2]) for i in range(n) for j in range(i+1,n)), "or3")
        model.addConstrs(((y_positions[j]+y_components_rotated[j] <= y_positions[i] + M*OR_overlap[i,j,3]) for i in range(n) for j in range(i+1,n)), "or4")
        model.addConstrs((gp.quicksum(OR_overlap[i,j,k] for k in range(4))<=3 for i in range(n) for j in range(n)), 'no_overlap_constraint') 

        # objective function
        model.setObjective(height, GRB.MINIMIZE)
        
        t1 = time.time()
        model.optimize()
        t2 = time.time() - t1
        if t2>timeout:
            return {}
        
        x_positions = []
        y_positions = []
        rota = []

        for i in range(n):
            x_positions.append(int(model.getVarByName(f'x_positions[{i}]').X))
            y_positions.append(int(model.getVarByName(f'y_positions[{i}]').X))
            rota.append(int(model.getVarByName(f'rotation[{i}]').X))
        
        rota = [False if rota[index]==0 else True for index in range(n)]
        tmp1 = [y_components[i] if rota[i] else x_components[i] for i in range(len(x_components))]
        y_components = [x_components[i] if rota[i] else y_components[i] for i in range(len(y_components))]
        x_components = tmp1
        
        # re-take the original order of the rectangles
        positions = [(x, y, i, z, k, q) for x, y, i, z, k, q in zip(x_positions, y_positions, x_components, y_components, rota, original_indices)]
        x_positions, y_positions, x_components, y_components, rota, _ = zip(*sorted(positions, key=lambda x: x[5]))
        
        
        height_found = int(model.ObjVal)
        return {'x_components':x_components,
                'y_components':y_components,
                'x_positions':x_positions,
                'y_positions':y_positions,
                'h':height_found,
                'time':t2,
                'W':w}
        
    except Exception as ex: print(ex)

In [ ]:
for i in tqdm(range(1,41)):
    file = LoadInstance('utils/instances/',f'ins-{i}.txt')  
    result = MIP(parameters=file, timeout=300, simmetry=True)
    if result:
        # if you want plot the results without saving them, use: PlotInstance(result)
        PlotInstance(result, 
                     path='MIP/images', 
                     name='rotation_fig-'+str(i))
        WriteInstance(result,
                     path='MIP/out',
                     name='rotation_out-'+str(i))
    else:
        print('Instance', i, 'TIMEOUT')